# LLMs com Hugging Face e LangChain

## Conteúdo

Neste notebook, exploraremos os fundamentos do trabalho com LLMs. Dividiremos nosso estudo em duas seções principais:

* **Hugging Face**:
    * Introdução à biblioteca `transformers`.
    * Uso de **Tokenizadores** para pré-processar texto.
    * Carregamento e utilização de **Modelos de Linguagem** pré-treinados para geração de texto.

* **LangChain**:
    * Introdução ao framework LangChain e ao conceito de **Runnables** com a LangChain Expression Language (LCEL).
    * Como instanciar e interagir com **LLMs** de provedores externos (ex: OpenAI).
    * Criação e formatação de **Prompts** e **Prompt Templates**.

# Hugging Face

O Hugging Face é uma empresa e uma comunidade de código aberto que se tornou o ecossistema central para o desenvolvimento de modelos de Processamento de Linguagem Natural (PLN). A sua principal biblioteca, `transformers`, fornece uma API unificada para acessar uma vasta gama de modelos pré-treinados, enquanto o *Model Hub* serve como um repositório para compartilhar modelos, datasets e demonstrações.

In [ ]:
# !pip install transformers

## Tokenizadores

O processo de tokenização consiste em converter uma sequência de texto (string) em uma sequência de tokens, que são subsequentemente mapeados para identificadores numéricos (IDs). Este passo é fundamental, pois os modelos de linguagem não operam sobre texto puro, mas sim sobre representações numéricas. Estratégias de tokenização sub-word, como Byte-Pair Encoding (BPE) ou WordPiece, são comuns, pois conseguem lidar com vocabulários extensos e palavras fora do vocabulário (Out-Of-Vocabulary - OOV) de forma eficiente.

In [ ]:
from transformers import AutoTokenizer

# Carregando um tokenizador pré-treinado do Hugging Face Hub.
# "gpt2" é um modelo causal (autoregressivo) da OpenAI.
tokenizer = AutoTokenizer.from_pretrained("gpt2")

Uma vez que o tokenizador é carregado, podemos utilizá-lo para codificar nosso texto de entrada. O resultado é um dicionário contendo, entre outras coisas, os `input_ids`, que são a representação numérica dos tokens, e a `attention_mask`, um tensor que indica ao modelo quais tokens devem ser considerados no cálculo da atenção (útil para processamento em lote com sentenças de tamanhos diferentes).

In [ ]:
text = "Artificial Intelligence is changing the world"

# Codificando o texto em IDs de input
encoded_input = tokenizer(text)

input_ids = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

print(f"Texto Original: {text}")
print(f"Input IDs: {input_ids}")
print(f"Attention Mask: {attention_mask}")

O processo inverso, a decodificação, converte a sequência de `input_ids` de volta para uma string legível. Isso é essencial para interpretar a saída gerada pelo modelo.

In [ ]:
# Decodificando os IDs de volta para texto
decoded_text = tokenizer.decode(input_ids)

print(f"Texto Decodificado: {decoded_text}")

## Modelos de Linguagem

A biblioteca `transformers` permite carregar modelos pré-treinados com a mesma simplicidade dos tokenizadores. Para tarefas de geração de texto, como a que estamos explorando, utilizamos classes como `AutoModelForCausalLM`. "Causal LM" refere-se a modelos de linguagem causal, que preveem o próximo token em uma sequência de forma autorregressiva.

In [ ]:
from transformers import AutoModelForCausalLM

# Carregando um modelo pré-treinado para linguagem causal ("text generation")
# Este modelo corresponde ao tokenizador "gpt2" que carregamos anteriormente
model = AutoModelForCausalLM.from_pretrained("gpt2")

Com o modelo e os inputs tokenizados em mãos, podemos realizar a inferência. O método `generate` é a principal interface para esta tarefa. Ele aceita uma variedade de parâmetros para controlar o processo de decodificação, como `max_length` (o comprimento máximo da sequência de saída) e `num_return_sequences` (o número de sequências independentes a serem geradas).

In [ ]:
# O método generate precisa de um tensor do PyTorch como entrada.
import torch

input_tensor = torch.tensor([input_ids])

# Gerando a continuação do texto
# Estamos pedindo ao modelo para gerar 50 tokens no total (input + output)
output_sequences = model.generate(
    input_tensor,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# A saída é uma lista de sequências de IDs
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

print("Texto Gerado pelo Modelo:")
print(generated_text)

# LangChain

LangChain é um framework projetado para simplificar o desenvolvimento de aplicações que utilizam LLMs. Ele fornece abstrações para componentes comuns, como modelos de linguagem, prompts e parsers de saída, e uma maneira declarativa de encadeá-los. O objetivo é facilitar a criação de aplicações complexas, como chatbots, sistemas de Resposta a Perguntas (Question Answering) e agentes autônomos.

In [ ]:
# Instalação das bibliotecas da LangChain e OpenAI
# !pip install langchain langchain-openai

## O Conceito de Runnables e LCEL

O coração do LangChain moderno é a LangChain Expression Language (LCEL). A LCEL fornece uma sintaxe declarativa para compor diferentes componentes. Qualquer objeto que segue o protocolo `Runnable` pode ser parte de uma cadeia LCEL. Este protocolo padroniza métodos como `invoke` (execução síncrona), `stream` (streaming da resposta) e `batch` (processamento em lote), unificando a interação com os componentes.

## LLMs em LangChain

LangChain oferece integrações com dezenas de provedores de modelos de linguagem. Para este exemplo, usaremos a integração com a OpenAI. É necessário possuir uma chave de API, que deve ser configurada como uma variável de ambiente para segurança.

In [ ]:
import os
from langchain_openai import ChatOpenAI

# É uma boa prática armazenar a chave de API como uma variável de ambiente.
# os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_API_AQUI"

# Instanciando o modelo de chat da OpenAI.
# O parâmetro "temperature" controla a aleatoriedade da saída.
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
os.environ["OPENAI_API_KEY"]

Após a instanciação, o modelo se comporta como um `Runnable` e pode ser invocado diretamente com o método `invoke`. A entrada para um modelo de chat é tipicamente uma lista de mensagens.

In [ ]:
from langchain_core.messages import HumanMessage

# Invocando o modelo com uma mensagem humana
response = llm.invoke([HumanMessage(content="Explique o que é a computação quântica em uma frase.")])

print(response)

In [ ]:
print(response.content)

## Prompts e Prompt Templates

Um prompt é a entrada enviada a um Modelo de Linguagem para instruí-lo a realizar uma tarefa. A engenharia de prompts é a disciplina que estuda como construir prompts eficazes. Em LangChain, os `PromptTemplate`s são objetos que facilitam a criação de prompts de forma dinâmica a partir de entradas do usuário. Eles contêm uma string de template e um conjunto de variáveis de entrada, permitindo a reutilização e a estruturação de interações complexas com o LLM.

In [ ]:
from langchain.prompts import PromptTemplate

# Criando um template com uma variável de entrada "topic"
prompt_template = PromptTemplate.from_template(
    "Explique o que é {topic} em uma frase."
)

# Formatando o template com um valor para a variável
formatted_prompt = prompt_template.format(topic="um buraco negro")

print(formatted_prompt)

Para modelos de chat, que são otimizados para conversação, a entrada não é uma única string, mas uma lista de mensagens. Cada mensagem possui um conteúdo e um "papel" (role), que define quem a enviou. O `ChatPromptTemplate` é a ferramenta utilizada para estruturar essa lista de mensagens de forma dinâmica.

### Papéis das Mensagens em Modelos de Chat

A estrutura de mensagens com papéis distintos é fundamental para controlar o comportamento dos modelos de chat. Os principais tipos são:

* **SystemMessage**: Esta mensagem estabelece o contexto, as instruções de alto nível, a persona ou as restrições para o comportamento do LLM. Geralmente, é a primeira mensagem na sequência e atua como uma diretriz para todas as interações subsequentes na mesma conversa. O modelo é fortemente influenciado por ela para definir seu tom, estilo e objetivo.

* **HumanMessage**: Representa a entrada do usuário final. É a mensagem que o LLM deve processar e à qual deve responder. Em uma aplicação, o conteúdo desta mensagem é tipicamente preenchido com a consulta do usuário.

* **AIMessage**: Representa uma resposta previamente gerada pelo próprio modelo de IA. O uso de `AIMessage`s no prompt é uma técnica poderosa conhecida como *few-shot prompting*. Ao fornecer exemplos de interações (pares de `HumanMessage` e `AIMessage`), podemos "ensinar" o modelo em tempo de inferência sobre o formato, o estilo ou o tipo de resposta esperada, sem a necessidade de re-treinamento.

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Template de chat com duas mensagens: uma de sistema e uma humana
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente de IA que explica conceitos científicos de forma simples e em apenas um parágrafo."),
    ("human", "Explique o que é {scientific_concept}.")
])

# Formatando o template de chat
formatted_chat_prompt = chat_template.format_messages(scientific_concept="a teoria da relatividade")

print(formatted_chat_prompt)

A seguir, um exemplo mais avançado que utiliza uma `AIMessage` para fornecer um exemplo de resposta ao modelo (*few-shot*), guiando-o para que as saídas futuras sigam um formato específico (neste caso, "Conceito: Explicação.").

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Exemplo de few-shot para guiar o formato da resposta
few_shot_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente que define termos técnicos. Responda sempre no formato 'Conceito: [explicação]'. Siga este formato estritamente."),
    ("human", "O que é uma Rede Neural?"), # Exemplo de pergunta
    ("ai", "Rede Neural: Um modelo computacional inspirado na estrutura e funcionamento do cérebro humano, utilizado para tarefas de aprendizado de máquina."), # Exemplo de resposta no formato desejado
    ("human", "{user_query}") # Pergunta real do usuário
])

# Formatando o template com uma nova consulta
few_shot_prompt = few_shot_template.format_messages(user_query="O que é Processamento de Linguagem Natural?")

print("Prompt Formatado Enviado ao Modelo:")
print(few_shot_prompt)

## Output Parsers

`OutputParser`s são componentes responsáveis por pegar a saída bruta de um LLM (geralmente uma string) e transformá-la em um formato mais estruturado e utilizável (ex: JSON, lista, etc.). O `StrOutputParser` é o mais simples: ele apenas garante que a saída seja uma string.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Instanciando o parser de string
string_parser = StrOutputParser()

## Encadeando Componentes com LCEL

Agora, podemos usar a sintaxe da LCEL para unir todos os componentes que vimos: `PromptTemplate`, `LLM` e `OutputParser`. O operador `|` passa o resultado do componente à esquerda como entrada para o componente à direita, criando um pipeline de processamento de dados.

In [ ]:
# Criando a cadeia (chain)
# A entrada (um dicionário) vai para o `chat_template`.
# A saída do template (um prompt formatado) vai para o `llm`.
# A saída do llm (uma mensagem de IA) vai para o `string_parser`.
# A saída do parser é o resultado final (uma string).
chain = chat_template | llm | string_parser

# Invocando a cadeia completa com uma única variável de entrada.
result = chain.invoke({"scientific_concept": "fissão nuclear"})

print(result)

Para ilustrar um parser mais complexo, vamos usar o `CommaSeparatedListOutputParser`, que instrui o modelo a retornar uma lista de itens separados por vírgula e, em seguida, converte essa string em uma lista Python.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# Instanciando o parser de lista separada por vírgulas
list_parser = CommaSeparatedListOutputParser()

# O template agora inclui instruções de formatação fornecidas pelo parser
list_prompt_template = PromptTemplate.from_template("Liste 5 exemplos de {category}. Sua resposta deve ser uma lista separada por vírgulas. Exemplo: foo, bar, baz")

# Criando a nova cadeia
list_chain = list_prompt_template | llm | list_parser

# Invocando a cadeia
list_result = list_chain.invoke({"category": "linguagens de programação"})

print(list_result)

### Exemplo Prático: Tradução

Vamos consolidar os conceitos anteriores em um exemplo prático. Criaremos uma cadeia de tradução que aceita como entrada o texto a ser traduzido, o idioma de origem e o idioma de destino. Isso demonstra a flexibilidade dos `PromptTemplates` para construir ferramentas reutilizáveis.

In [ ]:
# 1. Definição do ChatPromptTemplate com múltiplas variáveis de entrada
translation_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um tradutor poliglota altamente qualificado. Sua tarefa é traduzir o texto fornecido pelo usuário do idioma de origem para o idioma de destino com precisão."),
    ("human", "Por favor, traduza a seguinte frase de {source_language} para {target_language}: {text}")
])

In [ ]:
# 2. Construção da cadeia de tradução usando LCEL
translation_chain = translation_template | llm | StrOutputParser()

In [ ]:
# 3. Invocação da cadeia com os valores para as variáveis
translation_result = translation_chain.invoke({
    "source_language": "português",
    "target_language": "inglês",
    "text": "Inteligência artificial generativa permite a criação de novos conteúdos de forma autônoma."
})

print(f"Tradução: {translation_result}")

In [ ]:
translation_result_2 = translation_chain.invoke({
    "source_language": "inglês",
    "target_language": "espanhol",
    "text": "Large Language Models are transforming the way we interact with technology."
})

print(f"Tradução: {translation_result_2}")

### Geração de Saída Estruturada

Em muitas aplicações, receber uma resposta em formato de string não é suficiente. É preferível que a saída do LLM siga um esquema predefinido e estruturado, como um objeto JSON. Isso elimina a necessidade de analisar strings de forma manual e propensa a erros, permitindo que a saída do modelo seja diretamente utilizada como um objeto de dados ou uma estrutura de dados na aplicação.

LangChain facilita a geração de saídas estruturadas através da integração com a biblioteca **Pydantic**, que permite a definição de modelos de dados com tipagem estrita em Python.

O processo geral consiste em:

1.  **Definir um Schema**: Assim como antes, criamos uma classe que herda de `pydantic.BaseModel`, especificando os campos e os tipos de dados esperados.
2.  **Configurar o Modelo**: Chamamos `llm.with_structured_output(schema)` para criar uma nova versão do modelo que sempre retornará objetos com a estrutura do `schema`.
3.  **Encadear os Componentes**: A cadeia se torna mais simples: `prompt | modelo_estruturado`. Não há mais um parser explícito no final da cadeia.

O resultado da invocação será uma instância do nosso modelo Pydantic, com os dados já validados e tipados.

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    """Um modelo de dados que representa uma receita culinária."""
    name: str = Field(description="O nome da receita.")
    ingredients: List[str] = Field(description="Uma lista dos ingredientes necessários para a receita.")
    steps: List[str] = Field(description="Uma lista ordenada dos passos para preparar a receita.")
    preparation_time_minutes: int = Field(description="O tempo total de preparo em minutos.")

In [ ]:
structured_llm = llm.with_structured_output(Recipe)

In [ ]:
recipe_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um chef de cozinha experiente que cria receitas simples e claras."),
    ("human", "Por favor, gere uma receita para: {query}")
])

In [ ]:
structured_chain = recipe_prompt | structured_llm

In [ ]:
# Invocando a cadeia com uma consulta
user_query = "Pizza de Mussarela"
structured_result = structured_chain.invoke({"query": user_query})

# O resultado é um objeto Python, não uma string!
print(f"Tipo do Resultado: {type(structured_result)}")

In [ ]:
# Podemos acessar seus atributos diretamente
print(f"Nome da Receita: {structured_result.name}")
print(f"Tempo de Preparo: {structured_result.preparation_time_minutes} minutos\n")

print("Ingredientes:")
for ingredient in structured_result.ingredients:
    print(f"- {ingredient}")

print("\nPassos:")
for i, step in enumerate(structured_result.steps, 1):
    print(f"{i}. {step}")